In [255]:
import os
import gc
import pandas as pd
import numpy as np
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns

import time
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import xgboost as xgb

import warnings
warnings.filterwarnings("ignore")

np.random.seed(0)

In [256]:
train = pd.read_csv("train_fwYjLYX.csv", parse_dates=['application_date'])
test = pd.read_csv("test_1eLl9Yf.csv", parse_dates=['application_date'])
submission = pd.read_csv("sample_submission_IIzFVsf.csv", parse_dates=['application_date'])
train.shape,test.shape,submission.shape

((80402, 6), (180, 3), (180, 4))

In [257]:
test.drop(['id'], axis=1, inplace=True)
train = train.sort_values('application_date').reset_index(drop = True)
test = test.sort_values('application_date').reset_index(drop = True)

In [258]:
train.application_date.min(), train.application_date.max()

(Timestamp('2017-04-01 00:00:00'), Timestamp('2019-07-23 00:00:00'))

In [259]:
test.application_date.min(), test.application_date.max()

(Timestamp('2019-07-06 00:00:00'), Timestamp('2019-10-24 00:00:00'))

In [260]:
agg_func = {'case_count': ['sum']}
agg_name = train.groupby(['segment','application_date']).agg(agg_func)
agg_name.columns = [ 'SA_' + ('_'.join(col).strip()) for col in agg_name.columns.values]
agg_name.reset_index(inplace=True)
train = train.merge(agg_name, on=['segment','application_date'], how='left')
del agg_name
train.drop(['branch_id','state','zone','case_count'], axis=1, inplace=True)
train = train.rename(columns={'SA_case_count_sum': 'case_count'})
train.drop_duplicates(keep='first', inplace=True)
train = train.sort_values('application_date').reset_index(drop = True)
# df = train.append(test, ignore_index=True,sort=False)

train['train_or_test'] = 'train'
test['train_or_test'] = 'test'
df = pd.concat([train,test], sort=False)
print('Combined df shape:{}'.format(df.shape))
del train, test
gc.collect()

Combined df shape:(1830, 4)


201

In [261]:
df.head()

,application_date,segment,case_count,train_or_test
0,2017-04-01,1,299.0,train
1,2017-04-01,2,897.0,train
2,2017-04-02,2,605.0,train
3,2017-04-03,1,42.0,train
4,2017-04-03,2,2016.0,train


In [262]:
# Extracting application_date features
df['dayofmonth'] = df.application_date.dt.day
df['dayofyear'] = df.application_date.dt.dayofyear
df['dayofweek'] = df.application_date.dt.dayofweek
df['month'] = df.application_date.dt.month
df['year'] = df.application_date.dt.year
df['weekofyear'] = df.application_date.dt.weekofyear
df['is_month_start'] = (df.application_date.dt.is_month_start).astype(int)
df['is_month_end'] = (df.application_date.dt.is_month_end).astype(int)
df.head()

,application_date,segment,case_count,train_or_test,dayofmonth,dayofyear,dayofweek,month,year,weekofyear,is_month_start,is_month_end
0,2017-04-01,1,299.0,train,1,91,5,4,2017,13,1,0
1,2017-04-01,2,897.0,train,1,91,5,4,2017,13,1,0
2,2017-04-02,2,605.0,train,2,92,6,4,2017,13,0,0
3,2017-04-03,1,42.0,train,3,93,0,4,2017,14,0,0
4,2017-04-03,2,2016.0,train,3,93,0,4,2017,14,0,0


In [263]:
df.sort_values(by=['segment','application_date'], axis=0, inplace=True)
df.head(2)

,application_date,segment,case_count,train_or_test,dayofmonth,dayofyear,dayofweek,month,year,weekofyear,is_month_start,is_month_end
0,2017-04-01,1,299.0,train,1,91,5,4,2017,13,1,0
3,2017-04-03,1,42.0,train,3,93,0,4,2017,14,0,0


In [264]:
# Features constructed from previous case_count values

# Creating case_count lag features
def create_case_count_lag_feats(df, gpby_cols, target_col, lags):
    gpby = df.groupby(gpby_cols)
    for i in lags:
        df['_'.join([target_col, 'lag', str(i)])] = \
                gpby[target_col].shift(i).values + np.random.normal(scale=1.6, size=(len(df),))
    return df

# Creating case_count rolling mean features
def create_case_count_rmean_feats(df, gpby_cols, target_col, windows, min_periods=2, 
                             shift=1, win_type=None):
    gpby = df.groupby(gpby_cols)
    for w in windows:
        df['_'.join([target_col, 'rmean',str(shift), str(w)])] = \
            gpby[target_col].shift(shift).rolling(window=w, 
                                                  min_periods=min_periods,
                                                  win_type=win_type).mean().values + np.random.normal(scale=1.6, size=(len(df),))
    return df

# Creating case_count exponentially weighted mean features
def create_case_count_ewm_feats(df, gpby_cols, target_col, alpha=[0.9], shift=[1]):
    gpby = df.groupby(gpby_cols)
    for a in alpha:
        for s in shift:
            df['_'.join([target_col, 'lag', str(s), 'ewm', str(a)])] = \
                gpby[target_col].shift(s).ewm(alpha=a).mean().values + np.random.normal(scale=1.6, size=(len(df),))
    return df

In [265]:
# OHE of categorical features

def one_hot_encoder(df, ohe_cols=['dayofmonth','dayofweek','month','weekofyear']):
    '''
    One-Hot Encoder function
    '''
    print('Creating OHE features..\nOld df shape:{}'.format(df.shape))
    df = pd.get_dummies(df, columns=ohe_cols)
    print('New df shape:{}'.format(df.shape))
    return df

In [266]:
# Converting case_count to log(1+case_count)
df['case_count'] = np.log1p(df.case_count.values)
df.head(2)

,application_date,segment,case_count,train_or_test,dayofmonth,dayofyear,dayofweek,month,year,weekofyear,is_month_start,is_month_end
0,2017-04-01,1,5.703782,train,1,91,5,4,2017,13,1,0
3,2017-04-03,1,3.761200,train,3,93,0,4,2017,14,0,0


In [267]:
# Time-based Validation set

# For validation to keep months also identical to test set we can choose period (same of 2018) as the validation set.

masked_series = (df['application_date'] >= '2018-07-06') & (df['application_date'] <= '2018-10-24')
masked_series2 = (df['application_date'] < '2018-07-06') & (df['application_date'] > '2018-10-24')
df.loc[(masked_series), 'train_or_test'] = 'val'
df.loc[(masked_series2), 'train_or_test'] = 'no_train'
print('Train shape: {}'.format(df.loc[df.train_or_test=='train',:].shape))
print('Validation shape: {}'.format(df.loc[df.train_or_test=='val',:].shape))
print('No train shape: {}'.format(df.loc[df.train_or_test=='no_train',:].shape))
print('Test shape: {}'.format(df.loc[df.train_or_test=='test',:].shape))

Train shape: (1428, 12)
Validation shape: (222, 12)
No train shape: (0, 12)
Test shape: (180, 12)


In [268]:
# Model Validation

# Converting case_count of validation period to nan so as to resemble test period
train = df.loc[df.train_or_test.isin(['train','val']), :]
Y_val = train.loc[train.train_or_test=='val', 'case_count'].values.reshape((-1))
Y_train = train.loc[train.train_or_test=='train', 'case_count'].values.reshape((-1))
train.loc[train.train_or_test=='val', 'case_count'] = np.nan


train['mean_segment_dayofmonth'] = train.groupby(['segment', 'dayofmonth'])['case_count'].transform('mean') + np.random.normal(scale=1.6, size=(len(train),))
train['mean_segment_dayofyear'] = train.groupby(['segment', 'dayofyear'])['case_count'].transform('mean') + np.random.normal(scale=1.6, size=(len(train),))
train['mean_segment_dayofweek'] = train.groupby(['segment', 'dayofweek'])['case_count'].transform('mean') + np.random.normal(scale=1.6, size=(len(train),))
train['mean_segment_month'] = train.groupby(['segment', 'month'])['case_count'].transform('mean') + np.random.normal(scale=1.6, size=(len(train),))
train['mean_segment_weekofyear'] = train.groupby(['segment', 'weekofyear'])['case_count'].transform('mean') + np.random.normal(scale=1.6, size=(len(train),))


# # Creating case_count lag, rolling mean, rolling median, ohe features of the above train set
train = create_case_count_lag_feats(train, gpby_cols=['segment'], target_col='case_count', 
                                    lags=[91,98,105,112,119,126,182,364,546,728])

train = create_case_count_rmean_feats(train, gpby_cols=['segment'], 
                                 target_col='case_count', windows=[364,546], 
                                 min_periods=10, win_type='triang')

train = create_case_count_ewm_feats(train, gpby_cols=['segment'], 
                               target_col='case_count', 
                               alpha=[0.95, 0.9, 0.8, 0.7, 0.6, 0.5], 
                               shift=[91,98,105,112,119,126,182,364,546,728])

# One-Hot Encoding
train = one_hot_encoder(train, ohe_cols=['dayofweek','month']) 

# Final train and val datasets
val = train.loc[train.train_or_test=='val', :]
train = train.loc[train.train_or_test=='train', :]
print('Train shape:{}, Val shape:{}'.format(train.shape, val.shape))

Creating OHE features..
Old df shape:(1650, 89)
New df shape:(1650, 106)
Train shape:(1428, 106), Val shape:(222, 106)


In [269]:
avoid_cols = ['application_date', 'case_count', 'train_or_test', 'id', 'year','is_month_start']
cols = [col for col in train.columns if col not in avoid_cols]
print('No of training features: {} \nAnd they are:{}'.format(len(cols), cols))

No of training features: 101 
And they are:['segment', 'dayofmonth', 'dayofyear', 'weekofyear', 'is_month_end', 'mean_segment_dayofmonth', 'mean_segment_dayofyear', 'mean_segment_dayofweek', 'mean_segment_month', 'mean_segment_weekofyear', 'case_count_lag_91', 'case_count_lag_98', 'case_count_lag_105', 'case_count_lag_112', 'case_count_lag_119', 'case_count_lag_126', 'case_count_lag_182', 'case_count_lag_364', 'case_count_lag_546', 'case_count_lag_728', 'case_count_rmean_1_364', 'case_count_rmean_1_546', 'case_count_lag_91_ewm_0.95', 'case_count_lag_98_ewm_0.95', 'case_count_lag_105_ewm_0.95', 'case_count_lag_112_ewm_0.95', 'case_count_lag_119_ewm_0.95', 'case_count_lag_126_ewm_0.95', 'case_count_lag_182_ewm_0.95', 'case_count_lag_364_ewm_0.95', 'case_count_lag_546_ewm_0.95', 'case_count_lag_728_ewm_0.95', 'case_count_lag_91_ewm_0.9', 'case_count_lag_98_ewm_0.9', 'case_count_lag_105_ewm_0.9', 'case_count_lag_112_ewm_0.9', 'case_count_lag_119_ewm_0.9', 'case_count_lag_126_ewm_0.9', 'cas

In [270]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [271]:
train_x=train[cols]
test_x=val[cols]
train_y=Y_train
test_y=Y_val
train_x.shape, test_x.shape, train_y.shape, test_y.shape

((1428, 101), (222, 101), (1428,), (222,))

In [272]:
train_x.fillna(0,inplace=True)
test_x.fillna(0,inplace=True)

In [244]:
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
m=BaggingRegressor(n_estimators=100,random_state=0,n_jobs=-1,oob_score= True)
m.fit(train_x,train_y)
preds=m.predict(test_x)
predbr=np.expm1(preds)
print("MAPE: ",(mean_absolute_percentage_error(np.expm1(test_y),predbr)))

MAPE:  57.10979895883944


In [273]:
df['mean_segment_dayofmonth'] = df.groupby(['segment', 'dayofmonth'])['case_count'].transform('mean') + np.random.normal(scale=1.6, size=(len(df),))
df['mean_segment_dayofyear'] = df.groupby(['segment', 'dayofyear'])['case_count'].transform('mean') + np.random.normal(scale=1.6, size=(len(df),))
df['mean_segment_dayofweek'] = df.groupby(['segment', 'dayofweek'])['case_count'].transform('mean') + np.random.normal(scale=1.6, size=(len(df),))
df['mean_segment_month'] = df.groupby(['segment', 'month'])['case_count'].transform('mean') + np.random.normal(scale=1.6, size=(len(df),))
df['mean_segment_weekofyear'] = df.groupby(['segment', 'weekofyear'])['case_count'].transform('mean') + np.random.normal(scale=1.6, size=(len(df),))


# Creating case_count lag, rolling mean, rolling median, ohe features of the above train set
df_whole = create_case_count_lag_feats(df, gpby_cols=['segment'], target_col='case_count', 
                                  lags=[91,98,105,112,119,126,182,364,546,728])

df_whole = create_case_count_rmean_feats(df_whole, gpby_cols=['segment'], 
                                    target_col='case_count', windows=[364,546], 
                                    min_periods=10, win_type='triang')

df_whole = create_case_count_ewm_feats(df_whole, gpby_cols=['segment'], target_col='case_count', 
                                  alpha=[0.95, 0.9, 0.8, 0.7, 0.6, 0.5], 
                               shift=[91,98,105,112,119,126,182,364,546,728])

# One-Hot Encoding
df_whole = one_hot_encoder(df_whole, ohe_cols=['dayofweek','month'])

# Final train and test datasets
test = df_whole.loc[df_whole.train_or_test=='test', :]
train = df_whole.loc[~(df_whole.train_or_test=='test'), :]
print('Train shape:{}, Test shape:{}'.format(train.shape, test.shape))

Creating OHE features..
Old df shape:(1830, 89)
New df shape:(1830, 106)
Train shape:(1650, 106), Test shape:(180, 106)


In [274]:
train.shape, test.shape, train['case_count'].shape

((1650, 106), (180, 106), (1650,))

In [275]:
train_x=train[cols]
test_x=test[cols]
train_y=train['case_count']
train_x.shape, test_x.shape, train_y.shape

((1650, 101), (180, 101), (1650,))

In [276]:
train_x.fillna(0,inplace=True)
test_x.fillna(0,inplace=True)

In [277]:
m=BaggingRegressor(n_estimators=1000,random_state=0,n_jobs=-1,oob_score= True)
m.fit(train_x,train_y)
predbr=m.predict(test_x)
submission['case_count'] = np.expm1(predbr)
submission.to_csv('br_V31.csv', index=False)